<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3%E3%82%B5%E3%83%BC%E3%83%81%E5%88%A9%E6%B4%BB%E7%94%A8%E3%82%B9%E3%82%AD%E3%83%BC%E3%83%9E%E3%82%92%E7%94%A8%E3%81%84%E3%81%A6%E3%80%81%E6%97%A5%E8%8B%B1%E3%81%AE%E7%BF%BB%E8%A8%B3%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E4%BD%9C%E6%88%90%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ジャパンサーチ利活用スキーマを用いて、日英の翻訳データを作成する

In [6]:
import json
import urllib.parse
import requests
import time

###

# パラメータ

debug = True
unit = 10000
endpoint = "https://jpsearch.go.jp/rdf/sparql/"
output_file_path = "data.json"

###

page = 0
data = {}

start_all = time.time()

try:
    while 1:
        start = time.time()

        q = f"""PREFIX jps: <https://jpsearch.go.jp/term/property#>
PREFIX schema: <http://schema.org/>
PREFIX chname: <https://jpsearch.go.jp/entity/chname/>
SELECT DISTINCT ?label ?name WHERE {{
    ?cho rdfs:label ?label;
        schema:name ?name . 
    MINUS {{ ?cho jps:sourceInfo ?s . }}
    filter (lang(?name) = "en")  .
    filter (?label != str(?name)) . 
}} LIMIT {unit} OFFSET {unit * page}"""

        url = endpoint+"?query="+urllib.parse.quote(q)+"&format=json&output=json&results=json"

        if debug and page == 0:
            print("url", url)

        results = requests.get(url).json()
        if len(results["results"]["bindings"]) == 0:
            break

        page += 1

        if debug:
            print("page", page, ":", int(time.time() - start), "秒, ", "合計: ", int(time.time() - start_all), "秒")

        for i in range(len(results["results"]["bindings"])):
            
            obj = results["results"]["bindings"][i]

            label = obj["label"]["value"]
            name = obj["name"]["value"]

            data[label] = name

except Exception as e:
    with open(f"{output_file_path}_{page}.json", 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4,
        sort_keys=True, separators=(',', ': '))

with open(output_file_path, 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4,
    sort_keys=True, separators=(',', ': '))

url https://jpsearch.go.jp/rdf/sparql/?query=PREFIX%20jps%3A%20%3Chttps%3A//jpsearch.go.jp/term/property%23%3E%0APREFIX%20schema%3A%20%3Chttp%3A//schema.org/%3E%0APREFIX%20chname%3A%20%3Chttps%3A//jpsearch.go.jp/entity/chname/%3E%0ASELECT%20DISTINCT%20%3Flabel%20%3Fname%20WHERE%20%7B%0A%20%20%20%20%3Fcho%20rdfs%3Alabel%20%3Flabel%3B%0A%20%20%20%20%20%20%20%20schema%3Aname%20%3Fname%20.%20%0A%20%20%20%20MINUS%20%7B%20%3Fcho%20jps%3AsourceInfo%20%3Fs%20.%20%7D%0A%20%20%20%20filter%20%28lang%28%3Fname%29%20%3D%20%22en%22%29%20%20.%0A%20%20%20%20filter%20%28%3Flabel%20%21%3D%20str%28%3Fname%29%29%20.%20%0A%7D%20LIMIT%2010000%20OFFSET%200&format=json&output=json&results=json
page 1 : 2 秒,  合計:  2 秒
page 2 : 3 秒,  合計:  6 秒
page 3 : 3 秒,  合計:  9 秒
page 4 : 5 秒,  合計:  15 秒
page 5 : 7 秒,  合計:  23 秒
page 6 : 13 秒,  合計:  36 秒
page 7 : 12 秒,  合計:  49 秒
page 8 : 15 秒,  合計:  64 秒
page 9 : 16 秒,  合計:  81 秒
page 10 : 16 秒,  合計:  97 秒
